<a href="https://colab.research.google.com/github/D-Barradas/RAPIDS_HPO/blob/main/notebooks/HPO_cuML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HPO with dask-ml and cuml

## Introduction

&emsp; &emsp; &emsp; [Hyperparameter optimization](https://cloud.google.com/ai-platform/training/docs/hyperparameter-tuning-overview) is the task of picking the values for the hyperparameters of the model that provide the optimal results for the problem, as measured on a specific test dataset. This is often a crucial step and can help boost the model accuracy when done correctly. Cross-validation is often used to more accurately estimate the performance of the models in the search process. Cross-validation is the method of splitting the training set into complementary subsets and performing training on one of the subsets, then predicting the models performance on the other. This is a potential indication of how the model will generalise to data it has not seen before.

Despite its theoretical importance, HPO has been difficult to implement in practical applications because of the resources needed to run so many distinct training jobs.

The two approaches that we will be exploring in this notebook are :


#### 1. GridSearch

&emsp; &emsp; &emsp; As the name suggests, the "search" is done over each possible combination in a grid of parameters that the user provides. The user must manually define this grid.. For each parameter that needs to be tuned, a set of values are given and the final grid search is performed with tuple having one element from each set, thus resulting in a Catersian Product of the elements.

&emsp; &emsp; &emsp;For example, assume we want to perform HPO on XGBoost. For simplicity lets tune only `n_estimators` and `max_depth`

&emsp; &emsp; &emsp;`n_estimators: [50, 100, 150]`

&emsp; &emsp; &emsp;`max_depth: [6, 7, ,8]`
    
&emsp; &emsp; &emsp; The grid search will take place over |n_estimators| x |max_depth| which is 3 x 3 = 9. As you have probably guessed, the grid size grows rapidly as the number of parameters and their search space increases.

#### 2. RandomSearch


&emsp; &emsp; &emsp; [Random Search](http://www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf) replaces the exhaustive nature of the search from before with a random selection of parameters over the specified space. This method can outperform GridSearch in cases where the number of parameters affecting the model's performance is small (low-dimension optimization problems). Since this does not pick every tuple from the cartesian product, it tends to yield results faster, and the performance can be comparable to that of the Grid Search approach. It's worth keeping in mind that the random nature of this search means, the results with each run might differ.

Some of the other methods used for HPO include:

1. Bayesian Optimization

2. Gradient-based Optimization

3. Evolutionary Optimization

To learn more about HPO, some papers are linked to at the end of the notebook for further reading.

Now that we have a basic understanding of what HPO is, let's discuss what we wish to achieve with this demo. The aim of this notebook is to show the importance of hyper parameter optimisation and the performance of dask-ml GPU for xgboost and cuML-RF.

For this demo, we will be using the [Airline dataset](http://kt.ijs.si/elena_ikonomovska/data.html). The aim of the problem is to predict the arrival delay. It has about 116 million entries with 13 attributes that are used to determine the delay for a given airline. We have modified this problem to serve as a binary classification problem to determine if the airline will be delayed (True) or not.

Let's get started!

In [ ]:
import numpy as np
import pandas as pd

import cudf
import xgboost as xgb

from sklearn.metrics import make_scorer
from sklearn.utils import check_random_state
from sklearn.model_selection import KFold, ParameterSampler, RandomizedSearchCV, GridSearchCV


from cuml.ensemble import RandomForestClassifier
from cuml.model_selection import train_test_split
from cuml.metrics import accuracy_score

import time
import os
from urllib.request import urlretrieve
import gzip
import glob
from cuml.experimental.hyperopt_utils import plotting_utils

### Spinning up a CUDA Cluster

We start a local cluster and keep it ready for running distributed tasks with dask.


[LocalCUDACluster](https://github.com/rapidsai/dask-cuda) launches one Dask worker for each GPU in the current systems. It's developed as a part of the RAPIDS project.
Learn More:
- [Setting up Dask](https://docs.dask.org/en/latest/setup.html)
- [Dask Client](https://distributed.dask.org/en/latest/client.html)

## Mount the google Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Setup parameters  
NOTE: You must execute the download_data notebook before. It migth take up to 15 minutes

In [ ]:

num_rows = 2500000  # number of rows to be used in this notebook

# the parrent dir path is important to place the data and results in the correct flder
# parent_dir = "/".join(os.getcwd().split("/")[:-1])
parent_dir = "/content/drive/MyDrive/"

data_dir = os.path.join(parent_dir, "data", "airline-data")

orc_name = os.path.join(data_dir, "airline-data" + str(num_rows) + ".orc")

## Check the data is downloaded , otherwise execute the download_data.ipynb

In [ ]:
def prepare_dataset():
    try:
        if os.path.isfile(orc_name):
            df = cudf.read_orc(orc_name)
            df = df.drop("index",axis=1)
                # encode categoricals as numeric
            for col in df.select_dtypes(["object"]).columns:
                df[col] = df[col].astype("category").cat.codes.astype(np.int32)

            # cast all columns to int32
            for col in df.columns:
                df[col] = df[col].astype(np.float32)  # needed for random forest
            # reduce the size of the data for colab
            # we will use 10 K rows
            # df = df.head(10000)

            return df
    except FileNotFoundError:
        print(f"Error: Archive '{orc_name}' not found")

In [ ]:
df = prepare_dataset()

In [ ]:
type(df)

cudf.core.dataframe.DataFrame

In [ ]:
# Double check the precesence of Nan
for col in df.columns.to_list() :
    nan_vals = len ( df[df[col].isna()== True ])
    print (col , nan_vals)

ArrDelayBinary 0
YEAR 0
MONTH 0
DAY_OF_MONTH 0
DAY_OF_WEEK 0
CRS_DEP_TIME 0
CRS_ARR_TIME 0
OP_UNIQUE_CARRIER 0
OP_CARRIER_FL_NUM 0
ACTUAL_ELAPSED_TIME 0
ORIGIN 0
DEST 0
DISTANCE 0
DIVERTED 0


In [ ]:
import time
from contextlib import contextmanager
# Helping time blocks of code
@contextmanager
def timed(txt):
    t0 = time.time()
    yield
    t1 = time.time()
    print("%32s time:  %8.5f" % (txt, t1 - t0))

In [ ]:
# Define some default values to make use of across the notebook for a fair comparison
N_FOLDS = 3
N_ITER = 10  # The number of times that search combinations of parameters
SEED = check_random_state(73)

In [ ]:
label = 'ArrDelayBinary'

## Splitting Data

We split the data randomnly into train and test sets using the [cuml train_test_split](https://rapidsai.github.io/projects/cuml/en/0.12.0/api.html#cuml.preprocessing.model_selection.train_test_split) and create CPU versions of the data.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df, label, test_size=0.2)


#### Get the data into the CPU

In [ ]:
X_cpu = X_train.to_pandas()
y_cpu = y_train.to_numpy()

X_test_cpu = X_test.to_pandas()
y_test_cpu = y_test.to_numpy()

## Setup Custom cuML scorers

The search functions (such as GridSearchCV) for scikit-learn and dask-ml expect the metric functions (such as accuracy_score) to match the “scorer” API. This can be achieved using the scikit-learn's [make_scorer](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html) function.

We will generate a `cuml_scorer` with the cuML `accuracy_score` function.  You'll also notice an `accuracy_score_wrapper` which primarily converts the y label into a `float32` type. This is because some cuML models only accept this type for now and in order to make it compatible, we perform this conversion.

We also create helper functions for performing HPO in 2 different modes:
1. `gpu-grid`: Perform GPU based GridSearchCV
2. `gpu-random`: Perform GPU based RandomizedSearchCV

In [ ]:
def accuracy_score_wrapper(y, y_hat):
    """
        A wrapper function to convert labels to float32,
        and pass it to accuracy_score.

        Params:
        - y: The y labels that need to be converted
        - y_hat: The predictions made by the model
    """
    y = y.astype("float32") # cuML RandomForest needs the y labels to be float32
    return accuracy_score(y, y_hat, convert_dtype=True)

accuracy_wrapper_scorer = make_scorer(accuracy_score_wrapper)
cuml_accuracy_scorer = make_scorer(accuracy_score, convert_dtype=True)

#### This creates a wrapper to help us select the details of the HPO we want to compare and run

In [ ]:
def do_HPO(model, gridsearch_params, scorer, X, y, mode='gpu-Grid', n_iter=10):
    """
        Perform HPO based on the mode specified

        mode: default gpu-Grid. The possible options are:
        1. gpu-grid: Perform GPU based GridSearchCV
        2. gpu-random: Perform GPU based RandomizedSearchCV

        n_iter: specified with Random option for number of parameter settings sampled

        Returns the best estimator and the results of the search
    """
    if mode == 'gpu-grid':
        print("gpu-grid selected")
        clf = GridSearchCV(model,
                               gridsearch_params,
                               cv=N_FOLDS,
                               scoring=scorer)
    elif mode == 'gpu-random':
        print("gpu-random selected")
        clf = RandomizedSearchCV(model,
                               gridsearch_params,
                               cv=N_FOLDS,
                               scoring=scorer,
                               n_iter=n_iter)

    else:
        print("Unknown Option, please choose one of [gpu-grid, gpu-random]")
        return None, None
    res = clf.fit(X, y)
    print("Best clf and score {} {}\n---\n".format(res.best_estimator_, res.best_score_))
    return res.best_estimator_, res

In [ ]:


def manual_random_search_xgb(model, param_distributions, X, y, n_iter=10, n_splits=N_FOLDS, random_state=SEED):
    """
    Performs manual randomized search cross-validation for XGBoost using KFold.

    Args:
        model: The cuML/XGBoost model to train.
        param_distributions: Dictionary with parameters names (string) as keys and distributions
            or lists of parameters to sample from as values.
        X: cuDF DataFrame of features.
        y: cuDF Series of labels.
        n_iter: Number of parameter settings that are sampled.
        n_splits: Number of folds for cross-validation.
        random_state: Random state for reproducibility.

    Returns:
        A tuple containing the best estimator and a dictionary of results.
    """
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    param_sampler = ParameterSampler(param_distributions, n_iter=n_iter, random_state=random_state)

    best_score = -float('inf')
    best_params = None
    cv_results = []

    # Need to compute the splits once as they are dask delayed objects
    splits = list(kf.split(X))

    for i, params in enumerate(param_sampler):
        print(f"Fitting iteration {i+1}/{n_iter} with parameters: {params}")
        fold_scores = []

        for fold_idx, (train_idx, val_idx) in enumerate(splits):
            # Use cuDF indexing with computed indices
            # Compute the indices once per iteration to avoid recomputing in each fold
            train_idx_computed = train_idx
            val_idx_computed = val_idx

            X_train_fold = X.iloc[train_idx_computed]
            y_train_fold = y.iloc[train_idx_computed]
            X_val_fold = X.iloc[val_idx_computed]
            y_val_fold = y.iloc[val_idx_computed]

            # Ensure y is float32 for cuML
            y_train_fold = y_train_fold.astype('float32')
            y_val_fold = y_val_fold.astype('float32')


            # Train the model with current parameters
            current_model = model.set_params(**params)
            current_model.fit(X_train_fold, y_train_fold)

            # Evaluate on the validation fold
            y_pred_fold = current_model.predict(X_val_fold)
            score = accuracy_score(y_val_fold, y_pred_fold)
            fold_scores.append(score)

        mean_score = np.mean(fold_scores)
        cv_results.append({'params': params, 'mean_test_score': mean_score, 'fold_scores': fold_scores})

        if mean_score > best_score:
            best_score = mean_score
            best_params = params
            # Store the best estimator found during CV (optional, but can be useful)
            best_cv_estimator = current_model


    print("\n---")
    print(f"Best parameters found: {best_params}")
    print(f"Best cross-validation score: {best_score}")
    print("---")

    # Train the final model on the full training data with best params
    final_model = model.set_params(**best_params)
    final_model.fit(X, y.astype('float32')) # Ensure y is float32 for final training

    results_dict = {
        'best_estimator_': final_model,
        'best_score_': best_score,
        'best_params_': best_params,
        'cv_results_': cv_results
    }

    return final_model, results_dict

In [ ]:
def print_acc(model, X_train, y_train, X_test, y_test, mode_str="Default"):
    """
        Trains a model on the train data provided, and prints the accuracy of the trained model.
        mode_str: User specifies what model it is to print the value
    """
    y_pred = model.fit(X_train, y_train).predict(X_test)
    score = accuracy_score(y_pred, y_test.astype('float32'))
    print("{} model accuracy: {}".format(mode_str, score))

#### Look at trainin set shape, we are going to read 200k rows and 13 columns as features

In [ ]:
X_train.shape

(2000000, 13)

## Launch HPO

We will first see the model's performances without the gridsearch and then compare it with the performance after searching.

### XGBoost

To perform the Hyperparameter Optimization, we make use of the sklearn version of the [XGBClassifier](https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn).We're making use of this version to make it compatible and easily comparable to the scikit-learn version. The model takes a set of parameters that can be found in the documentation. We're primarily interested in the `max_depth`, `learning_rate`, `min_child_weight`, `reg_alpha` and `num_round` as these affect the performance of XGBoost the most.

Read more about what these parameters are useful for [here](https://xgboost.readthedocs.io/en/latest/parameter.html)

#### Default Performance

We first use the model with it's default parameters and see the accuracy of the model. In this case, it is 84%

In [ ]:
model_gpu_xgb_ = xgb.XGBClassifier(tree_method='hist',device = 'cuda',random_state=SEED)
print_acc(model_gpu_xgb_, X_train, y_cpu, X_test, y_test_cpu)

Default model accuracy: 0.848312


In [ ]:
# ## Random Forest
# model_rf_ = RandomForestClassifier()

# print("Default acc: ",accuracy_score(model_rf_.fit(X_train, y_train).predict(X_test), y_test))

#### Lets define a set of parameters to explore with model of interest

In [ ]:
# # For rf_model
# model_rf = RandomForestClassifier()

# # range
# params_rf = {
#     "max_depth": np.arange(start=3, stop = 15, step = 2), # Default = 6
#     "max_features": [0.1, 0.50, 0.75, 'auto'], #default = 0.3
#     "n_estimators": [100, 200, 1000]
#             }


#### Parameter Distributions

The way we define the grid to perform the search is by including ranges of parameters that need to be used for the search. In this example we make use of [np.arange](https://docs.scipy.org/doc/numpy/reference/generated/numpy.arange.html) which returns an ndarray of even spaced values, [np.logspace](https://docs.scipy.org/doc/numpy/reference/generated/numpy.logspace.html#numpy.logspace) returns a specified number of ssamples that are equally spaced on the log scale. We can also specify as lists, NumPy arrays or make use of any random variate sample that gives a sample when called. SciPy provides various functions for this too.

In [ ]:
# For xgb_model
model_gpu_xgb = xgb.XGBClassifier(tree_method='hist',device = 'cuda',random_state=SEED)

# More range
params_xgb = {
    "max_depth": np.arange(start=3, stop = 12, step = 3), # Default = 6
    "alpha" : np.logspace(-3, -1, 5), # default = 0
    "learning_rate": [0.05, 0.1, 0.15], #default = 0.3
    "min_child_weight" : np.arange(start=2, stop=10, step=3), # default = 1
    "n_estimators": [100, 200, 1000]
}

#### RandomizedSearchCV

We'll now try [RandomizedSearchCV](https://dask-ml.readthedocs.io/en/latest/modules/generated/dask_ml.model_selection.RandomizedSearchCV.html).
`n_iter` specifies the number of parameters points theat the search needs to perform. Here we will search `N_ITER` (defined earlier) points for the best performance.

In [ ]:
mode = "manual-gpu-random-xgb"

with timed("XGB-"+mode):
    # Use the manual random search function
    res, results = manual_random_search_xgb(model_gpu_xgb,
                                   params_xgb,
                                   X_train, # Pass cuDF DataFrame
                                   y_train, # Pass cuDF Series
                                   n_iter=N_ITER,
                                   n_splits=N_FOLDS,
                                   random_state=SEED)



Fitting iteration 1/10 with parameters: {'n_estimators': 1000, 'min_child_weight': np.int64(5), 'max_depth': np.int64(3), 'learning_rate': 0.05, 'alpha': np.float64(0.03162277660168379)}
Fitting iteration 2/10 with parameters: {'n_estimators': 200, 'min_child_weight': np.int64(8), 'max_depth': np.int64(3), 'learning_rate': 0.15, 'alpha': np.float64(0.03162277660168379)}
Fitting iteration 3/10 with parameters: {'n_estimators': 100, 'min_child_weight': np.int64(2), 'max_depth': np.int64(6), 'learning_rate': 0.05, 'alpha': np.float64(0.001)}
Fitting iteration 4/10 with parameters: {'n_estimators': 100, 'min_child_weight': np.int64(8), 'max_depth': np.int64(6), 'learning_rate': 0.05, 'alpha': np.float64(0.1)}
Fitting iteration 5/10 with parameters: {'n_estimators': 1000, 'min_child_weight': np.int64(2), 'max_depth': np.int64(9), 'learning_rate': 0.15, 'alpha': np.float64(0.03162277660168379)}
Fitting iteration 6/10 with parameters: {'n_estimators': 100, 'min_child_weight': np.int64(8), 'ma

In [ ]:
print_acc(model_gpu_xgb_, X_train, y_cpu, X_test, y_test_cpu)
print_acc(res, X_train, y_cpu, X_test, y_test_cpu, mode_str=mode)

Default model accuracy: 0.848312
manual-gpu-random-xgb model accuracy: 0.866868


In [ ]:
mode = "gpu-random"

with timed("XGB-"+mode):
    res, results = do_HPO(model_gpu_xgb,
                                   params_xgb,
                                   cuml_accuracy_scorer,
                                   X_train,
                                   y_cpu,
                                   mode=mode,
                                   n_iter=N_ITER)
print("Searched over {} parameters".format(len(results.cv_results_['mean_test_score'])))

gpu-random selected


/usr/local/lib/python3.11/dist-packages/distributed/client.py:3363: UserWarning: Sending large graph of size 144.08 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
INFO:distributed.scheduler:Client Client-05e436a4-785c-11f0-bf10-0242ac1c000c requests to retry 1 keys
INFO:distributed.scheduler:Client Client-05e436a4-785c-11f0-bf10-0242ac1c000c requests to retry 1 keys
INFO:distributed.scheduler:Client Client-05e436a4-785c-11f0-bf10-0242ac1c000c requests to retry 1 keys
INFO:distributed.scheduler:Client Client-05e436a4-785c-11f0-bf10-0242ac1c000c requests to retry 1 keys
INFO:distributed.scheduler:Client Client-05e436a4-785c-11f0-bf10-0242ac1c000c requests to retry 1 keys
INFO:distributed.scheduler:Client Client-05e436a4-785c-11f0-bf10-0242ac1c000c r

KeyboardInterrupt: 

#### XgBoost improves
With a very small set of parameter and with a random search we have a 0.863 accuracy vs 0.848 baseline accuracy a gain of 1% in about 2 minutes of testing, not bad. The main target for HPO is to explore wider ranges of parameters and usually with more data.

In [ ]:
# mode = "gpu-random"

# with timed("RF-"+mode):
#     res, results = do_HPO(model_rf,
#                           params_rf,
#                           cuml_accuracy_scorer,
#                           X_train.to_cupy().get(),
#                           y_cpu,
#                           mode=mode,
#                          n_iter = N_ITER)
# print("Searched over {} parameters".format(len(results.cv_results_['mean_test_score'])))

In [ ]:
# print("Improved acc: ",accuracy_score(res.predict(X_test), y_test))

#### Heatmaps
   - Between parameter pairs (we can do a combination of all possible pairs, but only one are shown in this notebook)
   - This gives a visual representation of how the pair affect the test score

In [ ]:
results.get("cv_results_")[0]

{'params': {'n_estimators': 100,
  'min_child_weight': np.int64(2),
  'max_depth': np.int64(6),
  'learning_rate': 0.05,
  'alpha': np.float64(0.1)},
 'mean_test_score': np.float64(0.8326195000000001),
 'fold_scores': [0.8331525, 0.8314175, 0.832685, 0.83334, 0.8325025]}

In [ ]:
df_randomsearch = pd.DataFrame(results[["cv_results_"],["params"]])
# plotting_utils.plot_heatmap(df_randomsearch, "max_depth", "n_estimators")

TypeError: unhashable type: 'list'

In [ ]:
df_randomsearch

,params,mean_test_score,fold_scores
0,"{'n_estimators': 100, 'min_child_weight': 2, '...",0.832620,"[0.8331525, 0.8314175, 0.832685, 0.83334, 0.83..."
1,"{'n_estimators': 200, 'min_child_weight': 5, '...",0.833179,"[0.8326, 0.8335475, 0.83363, 0.83281, 0.8333075]"
2,"{'n_estimators': 1000, 'min_child_weight': 8, ...",0.844618,"[0.84393, 0.845065, 0.8442675, 0.8446175, 0.84..."
3,"{'n_estimators': 200, 'min_child_weight': 5, '...",0.825242,"[0.82563, 0.8256275, 0.8253, 0.82478, 0.824875]"
4,"{'n_estimators': 100, 'min_child_weight': 8, '...",0.844419,"[0.84439, 0.8437175, 0.8437625, 0.8449325, 0.8..."


### Now lets perfrom the Grid-search strategy. This will explore all the the possible combinations , we should skip this for the sake of time (takes about 1 hrs)

In [ ]:
mode = "gpu-grid"

# For xgb_model
model_gpu_xgb = xgb.XGBClassifier(tree_method='gpu_hist',random_state=SEED)

with timed("XGB-"+mode):
    res, results = do_HPO(model_gpu_xgb,
                            params_xgb,
                                   cuml_accuracy_scorer,
                                   X_train,
                                   y_cpu,
                                   mode=mode)
print("Searched over {} parameters".format(len(results.cv_results_['mean_test_score'])))

In [ ]:
# mode = "gpu-grid"

# # For rf_model
# model_rf = RandomForestClassifier()

# with timed("RF-"+mode):
#     res, results = do_HPO(model_rf,
#                           params_rf,
#                           cuml_accuracy_scorer,
#                           X_train.to_cupy().get(),
#                           y_cpu,
#                           mode=mode,
#                          n_iter = N_ITER)
# print("Searched over {} parameters".format(len(results.cv_results_['mean_test_score'])))

In [ ]:
print_acc(model_gpu_xgb_, X_train, y_cpu, X_test, y_test_cpu)
print_acc(res, X_train, y_cpu, X_test, y_test_cpu, mode_str=mode)

#### We have run the classifiers with two strategies its up to you decide which one is the best
* **Xgboost baseline accuracy is 0.848**
* **Xgboost with Random search is 0.869 accuracy -> 5 minutes**
* **Xgboost with Grid search is 0.866 accuracy  -> 1 hour**

#### Mean/Std of test scores

We fix all parameters except one for each of these graphs and plot the effect the parameter has on the mean test score with the error bar indicating the standard deviation

In [ ]:
plotting_utils.plot_search_results(results)

In [ ]:
df_gridsearch = pd.DataFrame(results.cv_results_)
plotting_utils.plot_heatmap(df_gridsearch, "param_max_depth", "param_n_estimators")

## Conclusion and Next Steps

We notice improvements in the performance for a really basic version of the GridSearch and RandomizedSearch. Generally, the more data we use, the better the model performs, so you are encouraged to try for larger data and broader range of parameters.

This experiment can also be repeated with different classifiers and different ranges of parameters to notice how HPO can help improve the performance metric. In this example, we have chosen a basic metric - accuracy, but you can use more interesting metrics that help in determining the usefulness of a model. You can even send a list of parameters to the scoring function. This makes HPO really powerful, and it can add a significant boost to the model that we generate.


#### Further Reading

- [The 5 Classification Evaluation Metrics You Must Know](https://towardsdatascience.com/the-5-classification-evaluation-metrics-you-must-know-aa97784ff226)
- [11 Important Model Evaluation Metrics for Machine Learning Everyone should know](https://www.analyticsvidhya.com/blog/2019/08/11-important-model-evaluation-error-metrics/)
- [Algorithms for Hyper-Parameter Optimisation](http://papers.nips.cc/paper/4443-algorithms-for-hyper-parameter-optimization.pdf)
- [Forward and Reverse Gradient-Based Hyperparameter Optimization](http://proceedings.mlr.press/v70/franceschi17a/franceschi17a-supp.pdf)
- [Practical Bayesian Optimization of Machine
Learning Algorithms](http://papers.nips.cc/paper/4522-practical-bayesian-optimization-of-machine-learning-algorithms.pdf)
- [Random Search for Hyper-Parameter Optimization](http://jmlr.csail.mit.edu/papers/volume13/bergstra12a/bergstra12a.pdf)